In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/20 21:19:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Homework 5


**Question 1: 
Install Spark and PySpark**

Install Spark
Run PySpark
Create a local spark session
Execute spark.version.

What's the output?

3.3.2
2.1.4
1.2.3
5.4

**Answer:** 3.3.2

**Question 2:
HVFHW June 2021**

Read it with Spark using the same schema as we did in the lessons.
We will use this dataset for all the remaining questions.
Repartition it to 12 partitions and save it to parquet.
What is the average size of the Parquet (ending with .parquet extension) Files that were created (in MB)? Select the answer which most closely matches.

- 2MB
- 24MB
- 100MB
- 250MB

**Answer:** 55 MB

**Question 3:
Count records**

How many taxi trips were there on June 15?

Consider only trips that started on June 15.

- 308,164
- 12,856
- 452,470
- 50,982

In [2]:
df = spark.read \
    .option('header', 'true') \
    .parquet('fhvhv_tripdata_2021-06.parquet')

In [6]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [3]:
df = df.repartition(12)

In [4]:
df.registerTempTable('trips_data')

/home/maxwell5060/spark-3.3.2-bin-hadoop3/python/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [24]:
df_result = spark.sql("""
select count(1) from trips_data
where to_date(pickup_datetime) = '2021-06-15'
""")

In [25]:
df_result.show()

+--------+
|count(1)|
+--------+
|  452470|
+--------+



**Answer:** 452470

**Question 4:
Longest trip for each day**

Now calculate the duration for each trip.
How long was the longest trip in Hours?

- 66.87 Hours
- 243.44 Hours
- 7.68 Hours
- 3.32 Hours

In [30]:
df_result = spark.sql("""
select to_date(pickup_datetime), sum(trip_miles) from trips_data
group by to_date(pickup_datetime)
""")

In [31]:
df_result.show()

+------------------------+------------------+
|to_date(pickup_datetime)|   sum(trip_miles)|
+------------------------+------------------+
|              2021-06-22|2197405.2539999997|
|              2021-06-04|2523189.7269999995|
|              2021-06-20| 2641823.867999995|
|              2021-06-27|2693720.9590000007|
|              2021-06-28|2166664.5040000016|
|              2021-06-01|2065845.4230000004|
|              2021-06-17|       2470517.193|
|              2021-06-13|2648553.6839999994|
|              2021-06-19| 2970106.103999991|
|              2021-06-02|2186604.9149999963|
|              2021-06-08| 2122038.780000001|
|              2021-06-26|2934053.7440000055|
|              2021-06-14|2081557.1319999977|
|              2021-06-09|2254572.4190000035|
|              2021-06-29| 2196246.443999998|
|              2021-06-07|2094369.2080000006|
|              2021-06-06|2716537.8849999965|
|              2021-06-11|2623803.3680000007|
|              2021-06-18|2657914.

is trip_time EQUALS dropoff_datetime-pickup_datetime?

In [38]:
df_result = spark.sql("""
select (unix_timestamp(dropoff_datetime)-unix_timestamp(pickup_datetime)), trip_time from trips_data
""").show()

+--------------------------------------------------------------------------------------------------------------+---------+
|(unix_timestamp(dropoff_datetime, yyyy-MM-dd HH:mm:ss) - unix_timestamp(pickup_datetime, yyyy-MM-dd HH:mm:ss))|trip_time|
+--------------------------------------------------------------------------------------------------------------+---------+
|                                                                                                           656|      656|
|                                                                                                          1682|     1682|
|                                                                                                           332|      332|
|                                                                                                           645|      645|
|                                                                                                          1111|     1111|
|               

Now I'm ready to solve this puzzle:

In [42]:
df_result = spark.sql("""
select to_date(pickup_datetime) as date, max(trip_time/3600) as max_trip_time from trips_data
group by 1
order by 2 desc
""").show()

+----------+------------------+
|      date|     max_trip_time|
+----------+------------------+
|2021-06-25|  66.8788888888889|
|2021-06-22|25.549722222222222|
|2021-06-27|19.980833333333333|
|2021-06-26|18.197222222222223|
|2021-06-23|16.466944444444444|
|2021-06-24|13.909722222222221|
|2021-06-04|             11.67|
|2021-06-20|10.984444444444444|
|2021-06-01|           10.2675|
|2021-06-28| 9.966388888888888|
|2021-06-18| 9.624444444444444|
|2021-06-08| 9.480277777777777|
|2021-06-11| 9.471388888888889|
|2021-06-15| 9.402222222222223|
|2021-06-03| 9.365833333333333|
|2021-06-19| 9.106944444444444|
|2021-06-30| 9.056111111111111|
|2021-06-09| 9.030277777777778|
|2021-06-17| 8.774166666666666|
|2021-06-29| 8.571666666666667|
+----------+------------------+
only showing top 20 rows



**Answer:** 66.87 Hours

**Question 5:
User Interface**

Spark’s User Interface which shows application's dashboard runs on which local port?

- 80
- 443
- 4040
- 8080

**Answer:** 4040

**Question 6:
Most frequent pickup location zone**

Load the zone lookup data into a temp view in Spark

Using the zone lookup data and the fhvhv June 2021 data, what is the name of the most frequent pickup location zone?

- East Chelsea
- Astoria
- Union Sq
- Crown Heights North

In [5]:
zones = spark.read \
    .option('header', 'true') \
    .csv('taxi+_zone_lookup.csv')

In [6]:
zones.registerTempTable('zones')

In [49]:
df_result = spark.sql("""
select * from zones
limit 10
""").show()

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
+----------+-------------+--------------------+------------+



In [50]:
zones.printSchema()

root
 |-- LocationID: string (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [8]:
df_result = spark.sql("""
select zon.Zone as zon, count(*) from trips_data trd
left join zones zon on trd.PULocationID = zon.LocationID
group by 1
order by 2 desc
""").show()

+--------------------+--------+
|                 zon|count(1)|
+--------------------+--------+
| Crown Heights North|  231279|
|        East Village|  221244|
|         JFK Airport|  188867|
|      Bushwick South|  187929|
|       East New York|  186780|
|TriBeCa/Civic Center|  164344|
|   LaGuardia Airport|  161596|
|            Union Sq|  158937|
|        West Village|  154698|
|             Astoria|  152493|
|     Lower East Side|  151020|
|        East Chelsea|  147673|
|Central Harlem North|  146402|
|Williamsburg (Nor...|  143683|
|          Park Slope|  143594|
|  Stuyvesant Heights|  141427|
|        Clinton East|  139611|
|West Chelsea/Huds...|  139431|
|             Bedford|  138428|
|         Murray Hill|  137879|
+--------------------+--------+
only showing top 20 rows



**Answer:** Crown Heights North